In [ ]:
import gc
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np
import pandas as pd
import vectorbt as vbt
from scipy.stats import skew, kurtosis
from numba import njit


In [ ]:
portfolio_kwargs = dict(
    direction='longonly',
    freq='m',
)
# creamos el indicador para el lr y otro para el wlr
# lo hago por separado para poder calcular el mlr
# con data de varios activos #del mercado
# y luego solo utiliza lr con los que me interesa
@njit
def skew_kurtosis_nb(close):
    return skew(close, axis=1), kurtosis(close, axis=1)

SKEW_KURT = vbt.IndicatorFactory(
    input_names=['close'],
    output_names=['skew', 'kurtosis']
).from_apply_func(skew_kurtosis_nb, use_ray=True)

 #creamos el indicador para las bandas
@njit
def signals(skew, kurtosis, s_threshold, k_threshold ):
    signal = np.where(skew >= k_threshold and kurtosis <= -s_threshold, 1, 0)
    signal = np.where(skew <= k_threshold and kurtosis >= -s_threshold, -1, signal)
    return signal

SK_SIGNAL = vbt.IndicatorFactory(
    input_names=['skew', 'kurtosis'],
    param_names=['s_threshold', 'k_threshold'],
    output_names=['signal']
).from_apply_func(signals, use_ray=True)